In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)


In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [5]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [6]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [7]:
def calc_logloss(y, y_pred):    
    null_index = np.hstack((np.where(y_pred == 0), np.where(y_pred == 1)))
    if len(null_index) != 0:    
        y_pred = np.delete(y_pred,list(null_index[0]))
        y = np.delete(y,list(null_index[0]))    

    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальны

In [8]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

__Ошибка уменьшается с ростом параметра iterations при фиксированном параметре alpha=1e-1__

In [117]:
W = eval_model(X_st, y, iterations=1000000, alpha=1e-1)

1000000 [-92.65674077  -7.6664518  -26.48977581  73.04517435] 0.05865547841811411


In [121]:
W = eval_model(X_st, y, iterations=int(1e+7), alpha=1e-1)

10000000 [-221.2441203   -15.61810892  -65.53893671  169.31476273] 0.009881436998646981


In [9]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-5)

1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.2013133809011647


### 3.Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [10]:
def calc_pred_proba(W,X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W,X)))
    return y_pred_proba 

In [11]:
W = np.array([-221.2441203,   -15.61810892,  -65.53893671,  169.31476273])


In [12]:
calc_pred_proba(W ,X_st)

array([3.40198261e-02, 6.41012769e-14, 1.00000000e+00, 4.75133069e-08,
       9.98220802e-01, 5.24537576e-13, 1.00000000e+00, 3.28548750e-04,
       9.68072159e-01, 1.00000000e+00])

### 4.Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [33]:
def calc_pred(W,X):
    y_pred_proba = calc_pred_proba(W,X)
    y_pred = []
    for i in y_pred_proba:
        if (i > 0.5): 
            y_pred.append(1)
        elif (i <= 0.5):
            y_pred.append(0)
    return np.array(y_pred) 

In [34]:
y_pred = calc_pred(W ,X_st)

In [35]:
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

### Делаю сейчас

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [38]:
def accurac(y, y_pred):    
    return np.mean(np.abs(y - y_pred))
                   
accurac(y,y_pred)                  

0.0